In [0]:
from pyspark.sql.functions import trim, col, to_date
import datetime
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.functions import concat, col, lit
from pyspark.sql import functions as sf
from pyspark.sql.functions import countDistinct,count
from pyspark.sql.functions import max ,min
from pyspark.sql.functions import when
import pyspark.sql.functions as F 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.cluster.hierarchy as shc
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, date_sub, dayofweek
from pyspark.sql.functions import col, countDistinct
from pyspark.sql.functions import col, date_format, date_sub, expr
from pyspark.sql.functions import split, col
from pyspark.sql.functions import date_trunc
from pyspark.sql.functions import concat_ws,col
from pyspark.sql.functions import col, to_date
import datetime
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.functions import concat, col, lit
from pyspark.sql import functions as sf
from pyspark.sql.functions import countDistinct,count
from pyspark.sql.functions import max ,min
from pyspark.sql.functions import when
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum
from pyspark.sql import Window
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
sns.set()
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch
from sklearn.preprocessing import StandardScaler
import scipy.stats
from sklearn.model_selection import train_test_split
import random
from scipy import stats
from statsmodels.stats.weightstats import ztest
from scipy.stats import chisquare
import pandas as pd
from pyspark.sql.functions import regexp_replace, col

In [0]:
%pip install xgboost

In [0]:
%pip install shap

In [0]:
%pip install imbalanced-learn

## RX Rank

In [0]:
CATALOG_NAME = '`dsf-hackathon-hackathon-a-dev`'
spark.sql(f"USE CATALOG {CATALOG_NAME}")
rx_rank = spark.read.table(f'dsf_hackathon_2024.rx_rank')
rx_rank.display()
rx_rank.count()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, sum as sum_
 
df = rx_rank  
 
df_sorted = df.orderBy(col("pct").desc())

df_top_10_ndc = df_sorted.limit(10)

# Display the result

df_top_10_ndc.display()

## PX Rank

In [0]:
CATALOG_NAME = '`dsf-hackathon-hackathon-a-dev`'
spark.sql(f"USE CATALOG {CATALOG_NAME}")
px_rank = spark.read.table(f'dsf_hackathon_2024.px_rank')
px_rank.display()
px_rank.count()


df = px_rank   
df_sorted = df.orderBy(col("pct").desc())
df_top_10 = df_sorted.filter(col("pct") <= 100).limit(10)


df_top_10.display()

## Diagnosis Code

In [0]:
CATALOG_NAME = '`dsf-hackathon-hackathon-a-dev`'
spark.sql(f"USE CATALOG {CATALOG_NAME}")
diagnosis_codes = spark.read.table(f'dsf_hackathon_2024.dx_rank')
diagnosis_codes.display()
diagnosis_codes.select('diagnosis_code').distinct().count()


In [0]:
diagnosis_codes_top_10 = diagnosis_codes.orderBy(col("pct").desc()).filter(col("pct") >= 10).limit(10)
diagnosis_codes_top_10.display()

##Labs Data

In [0]:
CATALOG_NAME = '`dsf-hackathon-hackathon-a-dev`'
spark.sql(f"USE CATALOG {CATALOG_NAME}")
labs_data = spark.read.table(f'dsf_hackathon_2024.labs_play')
data_sample_labs_data = labs_data
data_sample_labs_data.display()
data_sample_labs_data.select('lab_test_loinc_cd').distinct().count()

##Demographic Data

In [0]:
CATALOG_NAME = '`dsf-hackathon-hackathon-a-dev`'
spark.sql(f"USE CATALOG {CATALOG_NAME}")
demog = spark.read.table(f'dsf_hackathon_2024.master_play')
demog.display()
demog.groupby('hiv').agg(F.count('id')).display()
data_sample_demog = demog
data_sample_demog.display()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, max as max_, first

df = data_sample_demog
 
# List of columns to be one-hot encoded
columns_to_encode = ["gender","region","race"]
 
# List of date columns (where we need to take the max date)
date_columns = []
 
# List of columns that are already numeric (0/1) or other types
numeric_columns = ['age']
 
# Track the dummy columns created
dummy_columns = []
 
# One-hot encode each categorical column
for column in columns_to_encode:
    distinct_values_df = df.select(column).distinct()
    for row in distinct_values_df.collect():
        value = row[column]
        encoded_col_name = f"{column}_{value}"
        dummy_columns.append(encoded_col_name)  # Track the new dummy column
        df = df.withColumn(encoded_col_name, when(col(column) == value, 1).otherwise(0))
 

aggregation_exprs = []
 
# Aggregate dummy columns using max
aggregation_exprs.extend([max_(col).alias(col) for col in dummy_columns])
 
# Aggregate date columns using max (to get the most recent date)
aggregation_exprs.extend([max_(col).alias(col) for col in date_columns])
 
# For numeric columns (0/1), we can use max or sum depending on the context
aggregation_exprs.extend([max_(col).alias(col) for col in numeric_columns])
 
# If there are other columns where you want to keep the first non-null value, you can do:
# aggregation_exprs.extend([first(col, ignorenulls=True).alias(col) for col in other_columns])
 
# Group by patient_id == id, hiv and apply the aggregations
df_data_sample_demog = df.groupBy("id",'hiv').agg(*aggregation_exprs)
 
df_data_sample_demog.display()


##Master Play Control Partition

In [0]:
CATALOG_NAME = '`dsf-hackathon-hackathon-a-dev`'
spark.sql(f"USE CATALOG {CATALOG_NAME}")
master_play_control = spark.read.table(f'dsf_hackathon_2024.master_play_control_partition')
master_play_control.display()
master_play_control.select('id').distinct().count()

In [0]:
master_play_control_top_10 = master_play_control.orderBy(col("pr").desc()).filter(col("pr") >= 90).limit(10)
master_play_control_top_10.display()

## Master Play HIV (Test) Partition

In [0]:
CATALOG_NAME = '`dsf-hackathon-hackathon-a-dev`'
spark.sql(f"USE CATALOG {CATALOG_NAME}")
master_play_hiv = spark.read.table(f'dsf_hackathon_2024.master_play_hiv_partition')
master_play_hiv.display()
master_play_hiv.select('id').distinct().count()

## Medical Claims Play

In [0]:
CATALOG_NAME = '`dsf-hackathon-hackathon-a-dev`'
spark.sql(f"USE CATALOG {CATALOG_NAME}")
medical_claims_play = spark.read.table(f'dsf_hackathon_2024.medical_claims_play')
medical_claims_play.display()


In [0]:
from pyspark.sql.functions import col, when

df1_diagnosis_codes = [row['diagnosis_code'] for row in diagnosis_codes_top_10.select("diagnosis_code").distinct().collect()]

medical_claims_play_with_top_10 = medical_claims_play.withColumn(
    "new_diagnosis_code",
    when(col("diagnosis_code").isin(df1_diagnosis_codes), col("diagnosis_code")).otherwise("others")
)

display(medical_claims_play_with_top_10)

df1_procedure_codes = [row['procedure_code'] for row in df_top_10.select("procedure_code").distinct().collect()]
medical_claims_play_with_top_10 = medical_claims_play_with_top_10.withColumn(
    "new_procedure_code",
    when(col("procedure_code").isin(df1_procedure_codes), col("procedure_code")).otherwise("others")
)

display(medical_claims_play_with_top_10)


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, max as max_, first

df = medical_claims_play_with_top_10
 
# List of columns to be one-hot encoded
columns_to_encode = ["new_diagnosis_code", "new_procedure_code"]
 
# List of date columns (where we need to take the max date)
date_columns = []
 
# List of columns that are already numeric (0/1) or other types
numeric_columns = []
 
# Track the dummy columns created
dummy_columns = []
 
# One-hot encode each categorical column
for column in columns_to_encode:
    distinct_values_df = df.select(column).distinct()
    for row in distinct_values_df.collect():
        value = row[column]
        encoded_col_name = f"{column}_{value}"
        dummy_columns.append(encoded_col_name)  # Track the new dummy column
        df = df.withColumn(encoded_col_name, when(col(column) == value, 1).otherwise(0))
 

aggregation_exprs = []
 
# Aggregate dummy columns using max
aggregation_exprs.extend([max_(col).alias(col) for col in dummy_columns])
 
# Aggregate date columns using max (to get the most recent date)
aggregation_exprs.extend([max_(col).alias(col) for col in date_columns])
 
# For numeric columns (0/1), we can use max or sum depending on the context
aggregation_exprs.extend([max_(col).alias(col) for col in numeric_columns])
 
# If there are other columns where you want to keep the first non-null value, you can do:
# aggregation_exprs.extend([first(col, ignorenulls=True).alias(col) for col in other_columns])
 
# Group by patient_id == id, hiv and apply the aggregations
df_grouped_master_claims = df.groupBy("id",'hiv').agg(*aggregation_exprs)
 
df_grouped_master_claims.display()


## Pharmacy Claims Play

In [0]:
CATALOG_NAME = '`dsf-hackathon-hackathon-a-dev`'
spark.sql(f"USE CATALOG {CATALOG_NAME}")
pharmacy_claims_play = spark.read.table(f'dsf_hackathon_2024.pharmacy_claims_play')
pharmacy_claims_play_sample = pharmacy_claims_play
pharmacy_claims_play_sample.display()

In [0]:
from pyspark.sql.functions import col, when

df1_ndc_codes = [row['ndc_code'] for row in df_top_10_ndc.select("ndc_code").distinct().collect()]

pharmacy_claims_play_with_top_10 = pharmacy_claims_play.withColumn(
    "new_ndc_code",
    when(col("ndc_code").isin(df1_ndc_codes), col("ndc_code")).otherwise("others")
)

display(pharmacy_claims_play_with_top_10)


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, max as max_, first

df = pharmacy_claims_play_with_top_10
 
# List of columns to be one-hot encoded
columns_to_encode = ["new_ndc_code"]
 
# List of date columns (where we need to take the max date)
date_columns = []
 
# List of columns that are already numeric (0/1) or other types
numeric_columns = []
 
# Track the dummy columns created
dummy_columns = []
 
# One-hot encode each categorical column
for column in columns_to_encode:
    distinct_values_df = df.select(column).distinct()
    for row in distinct_values_df.collect():
        value = row[column]
        encoded_col_name = f"{column}_{value}"
        dummy_columns.append(encoded_col_name)  # Track the new dummy column
        df = df.withColumn(encoded_col_name, when(col(column) == value, 1).otherwise(0))
 

aggregation_exprs = []
 
# Aggregate dummy columns using max
aggregation_exprs.extend([max_(col).alias(col) for col in dummy_columns])
 
# Aggregate date columns using max (to get the most recent date)
aggregation_exprs.extend([max_(col).alias(col) for col in date_columns])
 
# For numeric columns (0/1), we can use max or sum depending on the context
aggregation_exprs.extend([max_(col).alias(col) for col in numeric_columns])
 
# If there are other columns where you want to keep the first non-null value, you can do:
#aggregation_exprs.extend([first(col, ignorenulls=True).alias(col) for col in other_columns])
 
# Group by patient_id == id, hiv and apply the aggregations
df_grouped_pharmacy_claims_play = df.groupBy("id",'hiv').agg(*aggregation_exprs)
 
df_grouped_pharmacy_claims_play.display()


##Master Data

In [0]:
## Master Data Combined

join_key = "id" 
 
# Perform the left join between df1 and df2
df1_df2_joined = df_data_sample_demog.join(
    df_grouped_master_claims.select(join_key, *[col for col in df_grouped_master_claims.columns if col not in df_data_sample_demog.columns]),
    on=join_key,
    how="left"
)
 
# Now, join the result with df3
final_df_sample = df1_df2_joined.join(
    df_grouped_pharmacy_claims_play.select(join_key, *[col for col in df_grouped_pharmacy_claims_play.columns if col not in df1_df2_joined.columns]),
    on=join_key,
    how="left"
)

final_df_sample.display()

In [0]:
from pyspark.sql.functions import col, when

df_with_age_groups = final_df_sample.withColumn(
    "age_20_30", 
    when((col("age") >= 20) & (col("age") < 30), 1).otherwise(0)
).withColumn(
    "age_30_40", 
    when((col("age") >= 30) & (col("age") < 40), 1).otherwise(0)
).withColumn(
    "age_40_50", 
    when((col("age") >= 40) & (col("age") < 50), 1).otherwise(0)
).withColumn(
    "age_50_60", 
    when((col("age") >= 50) & (col("age") < 60), 1).otherwise(0)
).withColumn(
    "age_60_70", 
    when((col("age") >= 60) & (col("age") < 70), 1).otherwise(0)
).withColumn(
    "age_grater_than_70", 
    when((col("age") >= 70) & (col("age") < 100), 1).otherwise(0)
)

df_with_age_groups.display()

In [0]:
master_model_data = df_with_age_groups.drop('id','age').fillna(0)

master_model_data.display()

In [0]:
master_model_data.groupby('hiv').agg(F.count('hiv')).display()

In [0]:
pandas_df = master_model_data.toPandas()

### Model using RF and XGBoost Both

In [0]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from imblearn.over_sampling import SMOTE


# Separate features and target variable
X = pandas_df.drop(columns=["hiv"])
y = pandas_df["hiv"]

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

# Split the data into training and testing sets (70:30 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
 
# Define the parameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [300],
    'max_depth': [None],
    'min_samples_split': [5],
    'min_samples_leaf': [1],
    'max_features': ['auto']
}
 
# Define the parameter grid for XGBoost
param_grid_xgb = {
    'n_estimators': [300],
    'max_depth': [10],
    'learning_rate': [0.1],
    'subsample': [0.9],
    'colsample_bytree': [0.8]
}
 
# Initialize the models
rf = RandomForestClassifier(random_state=42)
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
 
# Perform GridSearchCV to find the best parameters for Random Forest
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=5, n_jobs=-1, verbose=2, scoring='f1')
grid_search_rf.fit(X_train, y_train)
 
# Perform GridSearchCV to find the best parameters for XGBoost
grid_search_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid_xgb, cv=5, n_jobs=-1, verbose=2, scoring='f1')
grid_search_xgb.fit(X_train, y_train)
 
# Get the best parameters and best models
best_params_rf = grid_search_rf.best_params_
best_model_rf = grid_search_rf.best_estimator_
 
best_params_xgb = grid_search_xgb.best_params_
best_model_xgb = grid_search_xgb.best_estimator_
 
print("Best Hyperparameters for Random Forest:", best_params_rf)
print("Best Hyperparameters for XGBoost:", best_params_xgb)
 
# Train the models with the best parameters
best_model_rf.fit(X_train, y_train)
best_model_xgb.fit(X_train, y_train)
 
# Predict on the test set
y_pred_rf = best_model_rf.predict(X_test)
y_pred_proba_rf = best_model_rf.predict_proba(X_test)[:, 1]
 
y_pred_xgb = best_model_xgb.predict(X_test)
y_pred_proba_xgb = best_model_xgb.predict_proba(X_test)[:, 1]
 
# Calculate F1 Score
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')
f1_xgb = f1_score(y_test, y_pred_xgb, average='weighted')
 
# Calculate ROC AUC
roc_auc_rf = roc_auc_score(y_test, y_pred_proba_rf)
roc_auc_xgb = roc_auc_score(y_test, y_pred_proba_xgb)
 
# Get feature importances
feature_importances_rf = best_model_rf.feature_importances_
feature_importances_xgb = best_model_xgb.feature_importances_

# Create DataFrames for feature importances
features_df_rf = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances_rf
}).sort_values(by='Importance', ascending=False)
 
features_df_xgb = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances_xgb
}).sort_values(by='Importance', ascending=False)


# Plot feature importances for Random Forest
plt.figure(figsize=(10, 16))
sns.barplot(x="Importance", y="Feature", data=features_df_rf, palette="viridis")
plt.title("Feature Importance - Random Forest")
plt.show()
 
# Plot feature importances for XGBoost
plt.figure(figsize=(10, 16))
sns.barplot(x="Importance", y="Feature", data=features_df_xgb, palette="viridis")
plt.title("Feature Importance - XGBoost")
plt.show()
 
# Plot ROC Curves
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_proba_rf)
roc_auc_value_rf = auc(fpr_rf, tpr_rf)
 
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_pred_proba_xgb)
roc_auc_value_xgb = auc(fpr_xgb, tpr_xgb)
 
plt.figure(figsize=(10, 8))
plt.plot(fpr_rf, tpr_rf, color='darkorange', lw=2, label=f'Random Forest ROC curve (area = {roc_auc_value_rf:.2f})')
plt.plot(fpr_xgb, tpr_xgb, color='blue', lw=2, label=f'XGBoost ROC curve (area = {roc_auc_value_xgb:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()
 
# Evaluate models on training and test sets
def evaluate_model(model, X_train, y_train, X_test, y_test):
    y_train_pred = model.predict(X_train)
    y_train_pred_proba = model.predict_proba(X_train)[:, 1]
 
    y_test_pred = model.predict(X_test)
    y_test_pred_proba = model.predict_proba(X_test)[:, 1]
 
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_precision = precision_score(y_train, y_train_pred)
    train_recall = recall_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred)
    train_roc_auc = roc_auc_score(y_train, y_train_pred_proba)
 
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred)
    test_recall = recall_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)
    test_roc_auc = roc_auc_score(y_test, y_test_pred_proba)
 
    print("\nTrain Set Metrics:")
    print(f"Accuracy: {train_accuracy:.4f}")
    print(f"Precision: {train_precision:.4f}")
    print(f"Recall: {train_recall:.4f}")
    print(f"F1 Score: {train_f1:.4f}")
    print(f"ROC AUC Score: {train_roc_auc:.4f}")
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_train, y_train_pred))
    print("\nClassification Report:")
    print(classification_report(y_train, y_train_pred))
 
    print("\nTest Set Metrics:")
    print(f"Accuracy: {test_accuracy:.4f}")
    print(f"Precision: {test_precision:.4f}")
    print(f"Recall: {test_recall:.4f}")
    print(f"F1 Score: {test_f1:.4f}")
    print(f"ROC AUC Score: {test_roc_auc:.4f}")
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_test_pred))
    print("\nClassification Report:")
    print(classification_report(y_test, y_test_pred))
 
# Evaluate Random Forest
print("Random Forest Evaluation:")
evaluate_model(best_model_rf, X_train, y_train, X_test, y_test)
 
# Evaluate XGBoost
print("XGBoost Evaluation:")
evaluate_model(best_model_xgb, X_train, y_train, X_test, y_test)

## Holdout set predictions

In [0]:
CATALOG_NAME = '`dsf-hackathon-hackathon-a-dev`'
spark.sql(f"USE CATALOG {CATALOG_NAME}")
labs_test = spark.read.table(f'dsf_hackathon_2024_holdout_participant.labs_test')
labs_test.display()
labs_test.count()

In [0]:
CATALOG_NAME = '`dsf-hackathon-hackathon-a-dev`'
spark.sql(f"USE CATALOG {CATALOG_NAME}")
master_test = spark.read.table(f'dsf_hackathon_2024_holdout_participant.master_test')
data_test_demog = master_test
data_test_demog.display()
data_test_demog.count()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, max as max_, first

df = data_test_demog
 
# List of columns to be one-hot encoded
columns_to_encode = ["gender","region","race"]
 
# List of date columns (where we need to take the max date)
date_columns = []
 
# List of columns that are already numeric (0/1) or other types
numeric_columns = ['age']
 
# Track the dummy columns created
dummy_columns = []
 
# One-hot encode each categorical column
for column in columns_to_encode:
    distinct_values_df = df.select(column).distinct()
    for row in distinct_values_df.collect():
        value = row[column]
        encoded_col_name = f"{column}_{value}"
        dummy_columns.append(encoded_col_name)  # Track the new dummy column
        df = df.withColumn(encoded_col_name, when(col(column) == value, 1).otherwise(0))
 

aggregation_exprs = []
 
# Aggregate dummy columns using max
aggregation_exprs.extend([max_(col).alias(col) for col in dummy_columns])
 
# Aggregate date columns using max (to get the most recent date)
aggregation_exprs.extend([max_(col).alias(col) for col in date_columns])
 
# For numeric columns (0/1), we can use max or sum depending on the context
aggregation_exprs.extend([max_(col).alias(col) for col in numeric_columns])
 
# If there are other columns where you want to keep the first non-null value, you can do:
# aggregation_exprs.extend([first(col, ignorenulls=True).alias(col) for col in other_columns])
 
# Group by patient_id == id, hiv and apply the aggregations
df_data_test_demog = df.groupBy("id").agg(*aggregation_exprs)
 
df_data_test_demog.display()


In [0]:
CATALOG_NAME = '`dsf-hackathon-hackathon-a-dev`'
spark.sql(f"USE CATALOG {CATALOG_NAME}")
medical_claims_test = spark.read.table(f'dsf_hackathon_2024_holdout_participant.medical_claims_test')
medical_claims_test.display()
medical_claims_test.count()

In [0]:
from pyspark.sql.functions import col, when

df1_diagnosis_codes = [row['diagnosis_code'] for row in diagnosis_codes_top_10.select("diagnosis_code").distinct().collect()]

medical_claims_test_with_top_10 = medical_claims_test.withColumn(
    "new_diagnosis_code",
    when(col("diagnosis_code").isin(df1_diagnosis_codes), col("diagnosis_code")).otherwise("others")
)

display(medical_claims_test_with_top_10)

df1_procedure_codes = [row['procedure_code'] for row in df_top_10.select("procedure_code").distinct().collect()]
medical_claims_test_with_top_10 = medical_claims_test_with_top_10.withColumn(
    "new_procedure_code",
    when(col("procedure_code").isin(df1_procedure_codes), col("procedure_code")).otherwise("others")
)

display(medical_claims_test_with_top_10)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, max as max_, first

df = medical_claims_test_with_top_10
 
# List of columns to be one-hot encoded
columns_to_encode = ["new_diagnosis_code", "new_procedure_code"]
 
# List of date columns (where we need to take the max date)
date_columns = []
 
# List of columns that are already numeric (0/1) or other types
numeric_columns = []
 
# Track the dummy columns created
dummy_columns = []
 
# One-hot encode each categorical column
for column in columns_to_encode:
    distinct_values_df = df.select(column).distinct()
    for row in distinct_values_df.collect():
        value = row[column]
        encoded_col_name = f"{column}_{value}"
        dummy_columns.append(encoded_col_name)  # Track the new dummy column
        df = df.withColumn(encoded_col_name, when(col(column) == value, 1).otherwise(0))
 

aggregation_exprs = []
 
# Aggregate dummy columns using max
aggregation_exprs.extend([max_(col).alias(col) for col in dummy_columns])
 
# Aggregate date columns using max (to get the most recent date)
aggregation_exprs.extend([max_(col).alias(col) for col in date_columns])
 
# For numeric columns (0/1), we can use max or sum depending on the context
aggregation_exprs.extend([max_(col).alias(col) for col in numeric_columns])
 
# If there are other columns where you want to keep the first non-null value, you can do:
# aggregation_exprs.extend([first(col, ignorenulls=True).alias(col) for col in other_columns])
 
# Group by patient_id == id, hiv and apply the aggregations
df_grouped_test_master_claims = df.groupBy("id").agg(*aggregation_exprs)
 
df_grouped_test_master_claims.display()

In [0]:
CATALOG_NAME = '`dsf-hackathon-hackathon-a-dev`'
spark.sql(f"USE CATALOG {CATALOG_NAME}")
pharmacy_claims_test = spark.read.table(f'dsf_hackathon_2024_holdout_participant.pharmacy_claims_test')
pharmacy_claims_test.display()
pharmacy_claims_test.count()

In [0]:
from pyspark.sql.functions import col, when

df1_ndc_codes = [row['ndc_code'] for row in df_top_10_ndc.select("ndc_code").distinct().collect()]

pharmacy_claims_test_with_top_10 = pharmacy_claims_test.withColumn(
    "new_ndc_code",
    when(col("ndc_code").isin(df1_ndc_codes), col("ndc_code")).otherwise("others")
)

display(pharmacy_claims_test_with_top_10)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, max as max_, first

df = pharmacy_claims_test_with_top_10
 
# List of columns to be one-hot encoded
columns_to_encode = ["new_ndc_code"]
 
# List of date columns (where we need to take the max date)
date_columns = []
 
# List of columns that are already numeric (0/1) or other types
numeric_columns = []
 
# Track the dummy columns created
dummy_columns = []
 
# One-hot encode each categorical column
for column in columns_to_encode:
    distinct_values_df = df.select(column).distinct()
    for row in distinct_values_df.collect():
        value = row[column]
        encoded_col_name = f"{column}_{value}"
        dummy_columns.append(encoded_col_name)  # Track the new dummy column
        df = df.withColumn(encoded_col_name, when(col(column) == value, 1).otherwise(0))
 

aggregation_exprs = []
 
# Aggregate dummy columns using max
aggregation_exprs.extend([max_(col).alias(col) for col in dummy_columns])
 
# Aggregate date columns using max (to get the most recent date)
aggregation_exprs.extend([max_(col).alias(col) for col in date_columns])
 
# For numeric columns (0/1), we can use max or sum depending on the context
aggregation_exprs.extend([max_(col).alias(col) for col in numeric_columns])
 
# If there are other columns where you want to keep the first non-null value, you can do:
#aggregation_exprs.extend([first(col, ignorenulls=True).alias(col) for col in other_columns])
 
# Group by patient_id == id, hiv and apply the aggregations
df_grouped_pharmacy_claims_test = df.groupBy("id").agg(*aggregation_exprs)
 
df_grouped_pharmacy_claims_test.display()

In [0]:
## Master Data Combined

join_key = "id" 
 
# Perform the left join between df1 and df2
df1_df2_joined_test = df_data_test_demog.join(
    df_grouped_test_master_claims.select(join_key, *[col for col in df_grouped_test_master_claims.columns if col not in df_data_test_demog.columns]),
    on=join_key,
    how="left"
)
 
# Now, join the result with df3
final_df_test = df1_df2_joined_test.join(
    df_grouped_pharmacy_claims_test.select(join_key, *[col for col in df_grouped_pharmacy_claims_test.columns if col not in df1_df2_joined_test.columns]),
    on=join_key,
    how="left"
)

final_df_test.display()

In [0]:
final_df_test.count()

In [0]:
from pyspark.sql.functions import col, when

df_test_with_age_groups = final_df_test.withColumn(
    "age_20_30", 
    when((col("age") >= 20) & (col("age") < 30), 1).otherwise(0)
).withColumn(
    "age_30_40", 
    when((col("age") >= 30) & (col("age") < 40), 1).otherwise(0)
).withColumn(
    "age_40_50", 
    when((col("age") >= 40) & (col("age") < 50), 1).otherwise(0)
).withColumn(
    "age_50_60", 
    when((col("age") >= 50) & (col("age") < 60), 1).otherwise(0)
).withColumn(
    "age_60_70", 
    when((col("age") >= 60) & (col("age") < 70), 1).otherwise(0)
).withColumn(
    "age_grater_than_70", 
    when((col("age") >= 70) & (col("age") < 100), 1).otherwise(0)
)

df_test_with_age_groups.display()

In [0]:
master_model_data_test = df_test_with_age_groups.drop('id','age').fillna(0)

pandas_df_test = master_model_data_test.toPandas()
display(pandas_df_test)
len(pandas_df_test.columns)

In [0]:
# Predict on the test set
y_pred_rf = best_model_rf.predict(pandas_df_test)
y_pred_proba_rf = best_model_rf.predict_proba(pandas_df_test)
 
y_pred_xgb = best_model_xgb.predict(pandas_df_test)
y_pred_proba_xgb = best_model_xgb.predict_proba(pandas_df_test)

In [0]:
pandas_df_test['y_pred_rf'] = y_pred_rf
pandas_df_test['y_pred_proba_rf'] = y_pred_proba_rf[:,1]
pandas_df_test['y_pred_xgb'] = y_pred_xgb
pandas_df_test['y_pred_proba_xgb'] = y_pred_proba_xgb[:,1]

display(pandas_df_test)

In [0]:
Final_test_with_predictions = df_test_with_age_groups.toPandas()

Final_test_with_predictions['y_pred_rf'] = y_pred_rf
Final_test_with_predictions['y_pred_proba_rf'] = y_pred_proba_rf[:,1]
Final_test_with_predictions['y_pred_xgb'] = y_pred_xgb
Final_test_with_predictions['y_pred_proba_xgb'] = y_pred_proba_xgb[:,1]


display(Final_test_with_predictions)

In [0]:
from datetime import datetime
Final_test_with_predictions['timestamp'] = datetime.now()
Final_test_with_predictions = Final_test_with_predictions[['id', 'y_pred_rf', 'y_pred_proba_rf', 'timestamp']]


In [0]:
Final_test_with_predictions = Final_test_with_predictions.rename(columns={'y_pred_rf': 'prediction', 'y_pred_proba_rf': 'positive_class_prob'})
display(Final_test_with_predictions)

In [0]:
scoring_dataset_hackathon_a_2024_08_30_10_46_47 = spark.createDataFrame(Final_test_with_predictions)

# 2024-08-30T10:46:47.263+00:00


In [0]:
scoring_dataset_hackathon_a_2024_08_30_10_46_47.count()

In [0]:
scoring_dataset_hackathon_a_2024_08_30_10_46_47.groupBy('prediction').count().show()

In [0]:
# Define the catalog, schema, and table name
catalog_name = "`dsf-hackathon-hackathon-a-dev`"  # Replace with your catalog name
schema_name = "`dsf-hackathon-scoring`"    # Replace with your schema name
table_name = "scoring_dataset_hackathon_a_2024_08_30_10_46_47"      # Replace with your table name

# Write the DataFrame as a table in the specified catalog and schema
scoring_dataset_hackathon_a_2024_08_30_10_46_47.write.format("delta").mode("overwrite").saveAsTable(f"{catalog_name}.{schema_name}.{table_name}")